In [4]:
%load_ext autoreload

%autoreload 2
import time

import pandas
import argparse

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions


from chainer.datasets import tuple_dataset
from chainer.dataset import convert, concat_examples

import numpy
import os
import glob

In [2]:
class MLP(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)    # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [1]:
print('hello')

hello


# データ読み込み

In [3]:
import easy_chainer

In [6]:
data, teach = easy_chainer.load_Data("C:/Users/Owner/Desktop/LAB/LAB 2019/解析/前処理/1_11/1_11.xlsx")
data = data.astype(numpy.float32)
teach = teach 
print(teach)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  7.  7.
  7.  7.  7.  7.  7.  7.  7.  7.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.
  9.  9.  9.  9.  9.  9.  9.  9.  9.  9. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]


C:\Users\Owner\Desktop\LAB\LAB 2018\jupyter File\easy_chainer.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = xls.as_matrix()[:-1]
C:\Users\Owner\Desktop\LAB\LAB 2018\jupyter File\easy_chainer.py:88: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  teach = xls.as_matrix()[-1]


In [7]:
teach = teach.astype(numpy.int8)
teach_bins = numpy.histogram(teach, bins=numpy.max(teach) + 1)
teach_bins

(array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], dtype=int64),
 array([ 0.        ,  0.91666667,  1.83333333,  2.75      ,  3.66666667,
         4.58333333,  5.5       ,  6.41666667,  7.33333333,  8.25      ,
         9.16666667, 10.08333333, 11.        ]))

In [8]:
k = 7


for i in range(int(max(teach) + 1)):
    tmp0_arr = numpy.where(teach == i)[0]
    try:
        tmp0 = numpy.random.choice(tmp0_arr, int(k), replace=False)
        if i == 0:
            id_train = tmp0
        else:
            id_train = numpy.hstack((id_train, tmp0))
    except Exception:
        print("Not teach")

In [9]:
id_train 

array([  5,   1,   3,   8,   0,   9,   7,  16,  17,  13,  14,  10,  18,
        11,  24,  20,  29,  27,  22,  23,  28,  35,  33,  30,  39,  34,
        37,  32,  45,  43,  41,  49,  44,  47,  42,  51,  54,  56,  52,
        50,  58,  53,  66,  63,  65,  68,  67,  69,  60,  75,  73,  78,
        72,  77,  76,  79,  87,  89,  85,  80,  84,  81,  83,  92,  90,
        98,  91,  97,  96,  93, 101, 106, 104, 108, 102, 100, 105, 118,
       117, 115, 114, 110, 112, 116], dtype=int64)

In [10]:
id_all = numpy.arange(1, len(teach) + 1, 1).astype(numpy.int32) - 1
# print(id_all)
id_test = numpy.delete(id_all, id_train)
print(id_test)

[  2   4   6  12  15  19  21  25  26  31  36  38  40  46  48  55  57  59
  61  62  64  70  71  74  82  86  88  94  95  99 103 107 109 111 113 119]


In [11]:
x_train, y_train = data[:, id_train], teach[id_train]
x_test, y_test = data[:, id_test], teach[id_test]

In [12]:
data = data.astype(numpy.float32)  # float64 ==> float32
print(data.shape)

(2385, 120)


In [13]:
train = tuple_dataset.TupleDataset(data[id_train],id_train.T)
train_iter = chainer.iterators.SerialIterator(train, 84)

test = tuple_dataset.TupleDataset(data[id_test], id_test.T)
test_iter = chainer.iterators.SerialIterator(test, 36, repeat=False, shuffle=False)

In [14]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

def parse_device(args):
    gpu = None
    if args.gpu is not None:
        gpu = args.gpu
    elif re.match(r'(-|\+|)[0-9]+$', args.device):
        gpu = int(args.device)

    if gpu is not None:
        if gpu < 0:
            return chainer.get_device(numpy)
        else:
            import cupy
            return chainer.get_device((cupy, gpu))

    return chainer.get_device(args.device)

In [15]:
net = MLP(1000, 12)

model = L.Classifier(predictor=net,
                     lossfun=F.softmax_cross_entropy,
                    accfun=F.accuracy)


In [16]:
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)
updater = training.updaters.StandardUpdater(
        train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (100, 'epoch'), out="Result2018_oono/%s" % time.strftime("%Y%m%d%H%M%S"))
out_directory = "./Result2018_oono/%s" % time.strftime("%Y%m%d_%H%M%S")

In [17]:
# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch
# frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
# trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Save two plot images to the result dir
trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='loss.png'))
trainer.extend(
    extensions.PlotReport(
        ['main/accuracy', 'validation/main/accuracy'],
        'epoch', file_name='accuracy.png'))

# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

# if args.resume:
#     # Resume from a snapshot
#     chainer.serializers.load_npz(args.resume, trainer)

# Run the training


In [18]:
trainer.run()



Exception in main training loop: index 54 is out of bounds for axis 0 with size 12
Traceback (most recent call last):
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\trainer.py", line 315, in run
    update()
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 165, in update
    self.update_core()
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 177, in update_core
    optimizer.update(loss_func, *in_arrays)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\optimizer.py", line 680, in update
    loss = lossfun(*args, **kwds)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\link.py", line 242, in __call__
    out = forward(*args, **kwargs)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\links\model\classifier.py", line 144, in forward
    self.loss = self.lossfun(self.y, t)
  File "C:\Users\Owner\Anaconda3\lib\site-packages\chainer\

IndexError: index 54 is out of bounds for axis 0 with size 12

In [ ]:
net.to_cpu()

chainer.serializers.save_hdf5("%s/model.hdf" % (out_directory),
                              obj=net)

numpy.save(arr=id_test, file="%s/test_id.npy" % (out_directory))
numpy.save(arr=id_train, file="%s/train_id.npy" % (out_directory))

In [21]:
net.to_cpu()

chainer.serializers.save_hdf5("%s/model.hdf" % (out_directory),
                              obj=net)

numpy.save(arr=id_test, file="%s/test_id.npy" % (out_directory))
numpy.save(arr=id_train, file="%s/train_id.npy" % (out_directory))

OSError: Unable to create file (unable to open file: name = './Result2018_oono/20190401_143925/model.hdf', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [23]:
test_iter.reset()

test_batch = test_iter.next()
test_spc, test_ref = concat_examples(test_batch)  # Test Dataset
for i in range(36):
    ref = test_ref[i]
    pred = numpy.argmax(net(test_spc[i].reshape(1, -1)).data)
    print("Ref: %d, Pred: %d %s" % (ref, pred, ref == pred))


Ref: 1, Pred: 0 False
Ref: 7, Pred: 0 False
Ref: 8, Pred: 0 False
Ref: 10, Pred: 0 False
Ref: 12, Pred: 0 False
Ref: 15, Pred: 0 False
Ref: 25, Pred: 0 False
Ref: 27, Pred: 0 False
Ref: 29, Pred: 0 False
Ref: 30, Pred: 0 False
Ref: 35, Pred: 0 False
Ref: 38, Pred: 0 False
Ref: 43, Pred: 0 False
Ref: 45, Pred: 0 False
Ref: 49, Pred: 0 False
Ref: 53, Pred: 0 False
Ref: 54, Pred: 0 False
Ref: 57, Pred: 0 False
Ref: 61, Pred: 0 False
Ref: 62, Pred: 0 False
Ref: 64, Pred: 0 False
Ref: 72, Pred: 0 False
Ref: 74, Pred: 0 False
Ref: 79, Pred: 0 False
Ref: 81, Pred: 0 False
Ref: 83, Pred: 0 False
Ref: 86, Pred: 0 False
Ref: 90, Pred: 0 False
Ref: 92, Pred: 0 False
Ref: 98, Pred: 0 False
Ref: 100, Pred: 0 False
Ref: 103, Pred: 0 False
Ref: 106, Pred: 0 False
Ref: 114, Pred: 0 False
Ref: 115, Pred: 0 False
Ref: 117, Pred: 0 False
